In [2]:
pip install efficientnet_pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=2bb1b04d68f7d5dd90da6b37c82787814d4e579b5b4c7498497cf8aa8f841118
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\13\d7\76\138efaee4cf27c8ac4837ee842a39f6574342e7038694cc409
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import cv2
import argparse

from efficientnet_pytorch import EfficientNet

In [7]:
brand_dict = {0:'Carnivla17-19', 1:'Carnivla20-21', 2:'Clubman17-20', 3:'Countryman17-21',4 :'Discovery17-20', 5:'ES17-20', 6:'Rangerover17-19',
            7:'Soul17-19', 8:'Sportage17-19', 9:'Sportage20-21'}
weight_path = r'C:\Users\user\Desktop\car_crush\091.차량 외관 영상 데이터\01.데이터\1.Training\brand\best_weights_b2_class_10.pth'
data_dir = r'C:\Users\user\Desktop\car_crush\091.차량 외관 영상 데이터\01.데이터\1.Training\test_image\C_210825_MI_089_17_RE_A_P_01_001.jpg'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

In [9]:
model = EfficientNet.from_pretrained('efficientnet-b2', num_classes=10)
model.load_state_dict(torch.load(weight_path, map_location=device))
model.to(device)

model.eval()

Loaded pretrained weights for efficientnet-b2


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [12]:
import numpy as np
img_array = np.fromfile(data_dir, np.uint8)
img = cv2.imdecode(img_array,cv2.IMREAD_COLOR)
inputs = cv2.resize(img, (200,200))
ori_img = inputs.copy()
inputs = inputs[:, :, ::-1].transpose((2, 0, 1)).copy()
inputs = torch.from_numpy(inputs).float().div(255.0).unsqueeze(0)
#inputs = inputs.cuda()

In [13]:
outputs = model(inputs)
_, preds = torch.max(outputs, 1)

cv2.putText(ori_img, str(brand_dict[preds.item()]),(75, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0,0,0))
cv2.imshow('result', ori_img)
cv2.waitKey(0)

-1